In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import lines as mlines
import os
from astropy.table import Table

## First load all the data we need into a table

We'll collate data both from a halo catalog snapshot an additionally different files storing halo history information. 

In [2]:
from read_sfh import get_scales
small_file_fname = "/Users/aphearin/Dropbox/UniverseMachine/data/histories/small_sfh_catalog_1.002310.txt"
scale_factor_array = get_scales(small_file_fname)

In [3]:
from assemble_catalog import assemble_catalog as assemble_history_data

snapshot_root_dirname = "/Users/aphearin/Dropbox/UniverseMachine/data/binary_reductions/z0/binaries/"
num_subvols = 144
history_cols = ('halo_id', 'halo_upid', 'halo_mpeak', 'halo_vmax_at_mpeak', 'halo_tidal_force', 
                'stellar_mass', 'sfr', 'sfr_mp', 'sm_mp')
history_data = Table(assemble_history_data(snapshot_root_dirname, num_subvols, *history_cols))


In [ ]:
def calculate_in_situ_ssfr_history(history_data, loc=-11.8, scale=0.3):
    nonzero_mask = history_data['sm_mp'] > 0.
    result = np.zeros_like(nonzero_mask).astype('f4')
    result[nonzero_mask] = history_data['sfr_mp'][nonzero_mask] / history_data['sm_mp'][nonzero_mask]
    result[~nonzero_mask] = np.nan
    result[np.isnan(result)] = 10**np.random.normal(loc=loc, scale=scale, size=len(result[np.isnan(result)]))
    return result

history_data['ssfr_history'] = calculate_in_situ_ssfr_history(history_data)
history_data['in_situ_quenched_history'] = history_data['ssfr_history'] < 10**-11

In [ ]:
from halocat_binary_reduction import assemble_halocat, read_column_info_array

halocat_binary_dirname = "/Users/aphearin/Dropbox/UniverseMachine/data/halocat_snapshot/a_1.00231"
column_info_fname = os.path.join(os.path.dirname(halocat_binary_dirname), 'column_info.dat')
column_info_array = read_column_info_array(column_info_fname)
propnames = ('halo_id', 'first_acc_scale', 'acc_rate_1tdyn', 'mpeak_scale', 'delta_vmax_tdyn_behroozi16', 
            'x', 'y', 'z', 'vx', 'vy', 'vz', 'mpeak')
halo_table = Table(assemble_halocat(halocat_binary_dirname, column_info_array, *propnames))
print("Available columns = \n{0}\n".format(halo_table.keys()))
print("number of halos in snapshot = {0}".format(len(halo_table)))

In [ ]:
from halotools.utils import crossmatch
idxA, idxB = crossmatch(history_data['halo_id'].data, halo_table['halo_id'].data)
cat = history_data[idxA]

keys_to_add = ('x', 'y', 'z', 'vx', 'vy', 'vz', 
               'first_acc_scale', 'acc_rate_1tdyn', 'mpeak_scale', 
               'delta_vmax_tdyn_behroozi16', 'mpeak')
for key in keys_to_add:
    cat[key] = halo_table[key][idxB]

In [ ]:
print("Percentage of histories with unmatched halos = {0:.3f}".format(
        len(history_data[idxA])/float(len(history_data))))

In [ ]:
print(cat.keys())

## SVM experimentation

In [ ]:
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)

keys_to_use = ('first_acc_scale', 'sfr_mp', 'sm_mp', 'delta_vmax_tdyn_behroozi16', 'mpeak_scale')
def val_to_sum(tpl):
        if len(tpl) <= 1:
            return 1
        else:
            return np.array(tpl[1:]).prod()

np.sum(list(val_to_sum(cat[key].shape) for key in keys_to_use))
    
# np.array(cat['sfr_mp'].shape[1:]).prod()

# np.sum(list(cat[key].shape[1:] for key in keys_to_use))

# clf.fit(digits.data[:-1], digits.target[:-1])  

In [ ]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()

In [ ]:
low_logsm, high_logsm = 10, 10.1
sample_mask = (cat['stellar_mass'] > 10**low_logsm) & (cat['stellar_mass'] < 10**high_logsm)
sample = cat[sample_mask]

In [ ]:
features = ('halo_delta_vmax_behroozi17', )
n_samples, n_features = len(sample), len(features)

X = np.zeros((n_samples, n_features))
for i, feature in enumerate(features):
    X[:, 0] = sample[feature]
y = sample['is_quenched'].data

In [ ]:
logit = logit.fit(X, y)

In [ ]:
plt.scatter(sample['halo_delta_vmax_behroozi17'], sample['is_quenched'])

In [ ]:
guess, confidence = logit.predict(X), logit.decision_function(X)

In [ ]:
np.mean(guess == y)

In [ ]:
confidence[0:10]

In [ ]:
__ = plt.hist(confidence, bins = np.linspace(-5, 5, 100))

In [ ]:
cat.keys()

In [ ]:
diff = cat['mpeak'] - cat['halo_mpeak']
fracdiff = diff/cat['halo_mpeak']

In [ ]:
__ = plt.hist(fracdiff, bins=np.linspace(-0.4, -0.3, 100), normed=True)

In [ ]:
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)

In [ ]:
X, y = 
clf.fit(digits.data[:-1], digits.target[:-1]) 

In [ ]:
np.random.normal?